# Start interactive job 

salloc --ntasks=1 --mem-per-cpu=8G --cpus-per-task=2 --time=00:30:00 --qos=devel --account=nn8014k

ml load Anaconda3/2022.10

conda activate /cluster/projects/nn8014k/magdalena/conda_environments/JupyterNotebook

jupyter notebook --port=8889

copy URL 

start code 



jupyter nbconvert --allow-errors --to notebook --execute /cluster/projects/nn8014k/magdalena/script_LncRNA/11_lncrna_rna_interaction/1_RIblast/lncRNA_interactions.txt

# After prediction  

After the prediction of interactiond (step 2) has completed the output has to be filtered. 

## Import dataframe

In [ ]:
import dask.dataframe as dd
import re

df = dd.read_csv('lncRNA_interactions.txt', skiprows=2, header=0)

# remove all white space around the column names of the dask dataframe
df.columns = df.columns.str.strip()

In [ ]:
def extract_and_calculate(row):
    numbers = list(map(int, re.findall(r'\d+', row)))
    diff1 = abs(numbers[1] - numbers[0])
    diff2 = abs(numbers[3] - numbers[2])
    return max(diff1, diff2)

df['BasePair'] = df['BasePair'].astype(str)
df[['num1', 'num2', 'num3', 'num4']] = df['BasePair'].str.extract(r'(\d+)-(\d+):(\d+)-(\d+)').astype(int)

df['difference_1'] = df['num2'] - df['num1']
df['difference_2'] = df['num4'] - df['num3']
df['counts'] = df[['difference_1', 'difference_2']].abs().max(axis=1)

result_df = df.compute()

In [ ]:
filtered = df.sort_values(by='Interaction Energy', ascending=False)
filtered = filtered.sort_values(by='counts', ascending=True)
filtered = filtered.drop_duplicates(subset='Query name', keep='first')

result_df = filtered.compute()

In [ ]:

# Specify the output file path and name
output_file = 'filtered.csv'

# Export the result_df Dask dataframe to a new CSV file
result_df.to_csv(output_file, index=False)